# Some tutorials
<a href="https://towardsdatascience.com/object-detection-explained-r-cnn-a6c813937a76"> region proposal using r-cnn</a>

<a href="https://learnopencv.com/selective-search-for-object-detection-cpp-python/"> region proposal using selective segmentation</a>

https://github.com/chingisooinar/Object-Detection_from-Scratch/blob/main/RCNN/RCNN.ipynb

<a href="https://towardsdatascience.com/step-by-step-r-cnn-implementation-from-scratch-in-python-e97101ccde55">RCNN-implemenation</a>

# Selective selection

In [1]:
cd

/home/dhruv


In [2]:
pwd

'/home/dhruv'

In [3]:
cd drone-defense-wall/data

/home/dhruv/drone-defense-wall/data


In [4]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [5]:
#!/usr/bin/env python
'''
Usage:
./ssearch.py input_image (f|q)
f=fast, q=quality
Use "l" to display less rects, 'm' to display more rects, "q" to quit.
'''

import sys
import cv2


# If image path and f/q is not passed as command
# line arguments, quit and display help message
if len(sys.argv) < 3:
    print(__doc__)
    sys.exit(1)

# speed-up using multithreads
cv2.setUseOptimized(True);
cv2.setNumThreads(4);

# read image
im = cv2.imread("194.jpg")
# resize image
newHeight = 200
newWidth = int(im.shape[1]*200/im.shape[0])
im = cv2.resize(im, (newWidth, newHeight))    

# create Selective Search Segmentation Object using default parameters
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

# set input image on which we will run segmentation
ss.setBaseImage(im)

# Switch to fast but low recall Selective Search method
#if (sys.argv[2] == 'f'):
ss.switchToSelectiveSearchFast()

# Switch to high recall but slow Selective Search method
# elif (sys.argv[2] == 'q'):
#     ss.switchToSelectiveSearchQuality()
# # if argument is neither f nor q print help message
# else:
#     print(__doc__)
#     sys.exit(1)

# run selective search segmentation on input image
rects = ss.process()

# taking only the upper 2/3rd of the image
bboxes=[]
for x,y,w,h in rects:
    if y<100:
        bboxes.append([x,y,w,h])
bboxes=np.array(bboxes)
rects=bboxes
print('Total Number of Region Proposals: {}'.format(len(rects)))

# number of region proposals to show
numShowRects = 80
# increment to increase/decrease total number
# of reason proposals to be shown
increment = 10





while True:
    # create a copy of original image
    imOut = im.copy()

    # itereate over all the region proposals
    for i, rect in enumerate(rects):
        # draw rectangle for region proposal till numShowRects
        if (i < numShowRects):
            x, y, w, h = rect
            cv2.rectangle(imOut, (x, y), (x+w, y+h), (0, 255, 0), 1, cv2.LINE_AA)


        else:
            break

    # show output
    cv2.imshow("Output", imOut)

    # record key press
    k = cv2.waitKey(0) & 0xFF

    # m is pressed
    if k == 109:
        # increase total number of rectangles to show by increment
        numShowRects += increment
    # l is pressed
    elif k == 108 and numShowRects > increment:
        # decrease total number of rectangles to show by increment
        numShowRects -= increment
    # q is pressed
    elif k == 113:
        break
# close image show window
cv2.destroyAllWindows()


Total Number of Region Proposals: 119


# Object Tracking
- using boosting and median tracking

In [2]:
cd

/home/dhruv


In [2]:
cd openCV-practice/Computer-Vision-with-Python/

[Errno 2] No such file or directory: 'openCV-practice/Computer-Vision-with-Python/'
/home/dhruv/drone-defense-wall/notebooks


In [7]:
def ask_for_tracker():
    print("Welcome! What Tracker API would you like to use?")
    print("Enter 0 for BOOSTING: ")
    print("Enter 1 for MIL: ")
    print("Enter 2 for KCF: ")
    print("Enter 3 for TLD: ")
    print("Enter 4 for MEDIANFLOW: ")
    choice = 3
    
    if choice == '0':
        tracker = cv2.legacy.TrackerBoosting_create()
    if choice == '1':
        tracker = cv2.legacy.TrackerMIL_create()
    if choice == '2':
        tracker = cv2.legacy.TrackerKCF_create()
    if choice == '3':
        tracker = cv2.legacy.TrackerTLD_create()
    if choice == '4':
        tracker = cv2.legacy.TrackerMedianFlow_create()


    return tracker
 



In [6]:
import cv2
tracker = ask_for_tracker()
tracker_name = str(tracker).split()[0][1:]

# Read video
cap = cv2.VideoCapture(0)

# Read first frame.
ret, frame = cap.read()


# Special function allows us to draw on the very first frame our desired ROI
## replace this roi by the coordinates from the resultant image from the DL model resultant
roi = cv2.selectROI(frame, False)

# Initialize tracker with first frame and bounding box
ret = tracker.init(frame, roi)

while True:
    # Read a new frame
    ret, frame = cap.read()
    
    
    # Update tracker
    success, roi = tracker.update(frame)
    
    # roi variable is a tuple of 4 floats
    # We need each value and we need them as integers
    (x,y,w,h) = tuple(map(int,roi))
    
    # Draw Rectangle as Tracker moves
    if success:
        # Tracking success
        p1 = (x, y)
        p2 = (x+w, y+h)
        cv2.rectangle(frame, p1, p2, (0,255,0), 3)
        print(f'{p1} and {p2}')
    else :
        # Tracking failure
        cv2.putText(frame, "Failure to Detect Tracking!!", (100,200), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255),3)

    # Display tracker type on frame
    cv2.putText(frame, tracker_name, (20,400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),3);

    # Display result
    cv2.imshow(tracker_name, frame)

    # Exit if ESC pressed
    k = cv2.waitKey(1) & 0xff
    if k == 27 : 
        break
        
cap.release()
cv2.destroyAllWindows()

Welcome! What Tracker API would you like to use?
Enter 0 for BOOSTING: 
Enter 1 for MIL: 
Enter 2 for KCF: 
Enter 3 for TLD: 
Enter 4 for MEDIANFLOW: 
Please select your tracker: 3
(185, 134) and (288, 262)
(185, 134) and (288, 262)
(185, 135) and (288, 264)
(185, 134) and (288, 262)
(175, 185) and (278, 313)
(175, 185) and (278, 313)
(176, 185) and (279, 314)
(176, 187) and (278, 315)
(176, 187) and (278, 315)
(176, 186) and (278, 314)
(176, 187) and (279, 315)
(177, 188) and (279, 316)
(177, 187) and (279, 314)
(178, 186) and (280, 314)
(179, 185) and (281, 313)
(180, 184) and (283, 312)
(180, 182) and (283, 310)
(179, 181) and (282, 310)
(160, 111) and (283, 265)
(158, 112) and (281, 266)
(155, 112) and (277, 264)
(151, 111) and (272, 262)
(147, 112) and (266, 261)
(154, 165) and (257, 293)
(150, 166) and (252, 293)
(145, 167) and (247, 294)
(140, 166) and (242, 293)
(134, 165) and (237, 293)
(129, 165) and (231, 292)
(123, 123) and (226, 251)
(118, 122) and (221, 251)
(113, 119) an

## To-do modifying the above code to take in corrdinates from the DL model resultant image

In [10]:
type(roi)

tuple

In [1]:
# modified code
import cv2
tracker = ask_for_tracker()
tracker_name = str(tracker).split()[0][1:]

# Read video
cap = cv2.VideoCapture("DATA/cam0.mp4")

# Read first frame.
ret, frame = cap.read()


# Special function allows us to draw on the very first frame our desired ROI
## replace this roi by the coordinates from the resultant image from the DL model resultant
roi = (499.0, 305.0, 54.0, 57.0)
# Initialize tracker with first frame and bounding box
ret = tracker.init(frame, roi)

while True:
    # Read a new frame
    ret, frame = cap.read()
    
    
    # Update tracker
    success, roi = tracker.update(frame)
    
    # roi variable is a tuple of 4 floats
    # We need each value and we need them as integers
    (x,y,w,h) = tuple(map(int,roi))
    
    # Draw Rectangle as Tracker moves
    if success:
        # Tracking success
        p1 = (x, y)
        p2 = (x+w, y+h)
        cv2.rectangle(frame, p1, p2, (0,255,0), 3)
        print(f'{p1} and {p2}')
    else :
        # Tracking failure
        cv2.putText(frame, "Failure to Detect Tracking!!", (100,200), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255),3)

    # Display tracker type on frame
    cv2.putText(frame, tracker_name, (20,400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),3);

    # Display result
    cv2.imshow(tracker_name, frame)

    # Exit if ESC pressed
    k = cv2.waitKey(1) & 0xff
    if k == 27 : 
        break
        
cap.release()
cv2.destroyAllWindows()

NameError: name 'ask_for_tracker' is not defined

In [11]:
roi

(499.0, 305.0, 54.0, 57.0)